In [ ]:
import holoviews as hv

hv.extension('bokeh')

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

from torch.autograd import Variable
import torch
import torch.nn as nn


from lib.util import output_to_xr, dict_to_xr
from lib.torch_models import TorchRegressor, single_layer_perceptron, residual_net
from lib.plots.model_evaluation import scatter_plot_z



Open NGaqua data

define the loss, which should be mass-weighted

In [ ]:
data = joblib.load("../data/ml/ngaqua/data.pkl")

# load weight data
_, w = data['w']
w = Variable(torch.FloatTensor(w))

# mass weighted loss
mse = nn.MSELoss()

def loss_function(output, y):
    return mse(output.mul(w.sqrt()), y.mul(w.sqrt()))

In [ ]:
x_train, y_train = data['train']

inds = np.random.choice(x_train.shape[0], 200000)
x_train, y_train = x_train[inds], y_train[inds]

In [ ]:
slp = TorchRegressor(net_fn=single_layer_perceptron,
               loss_fn=loss_function,
               optim_kwargs={'lr': .001},
                    num_epochs=2)

slp = make_pipeline(VarianceThreshold(.001), StandardScaler(), slp)

slp.fit(x_train, y_train.data)

In [ ]:
rnet = TorchRegressor(net_fn=residual_net,
               loss_fn=loss_function,
               optim_kwargs={'lr': .001},
                     num_epochs=2)

rnet = make_pipeline(VarianceThreshold(.001), StandardScaler(), rnet)
rnet.fit(x_train, y_train.data)

In [ ]:
lm = make_pipeline(VarianceThreshold(.001), StandardScaler(), LinearRegression())
lm.fit(x_train, y_train)

In [ ]:
import pandas as pd

x, y = data['test']

predictions = {'true': y, 'rnet': rnet.predict(x), 'slp': slp.predict(x), 'linear': lm.predict(x)}
predictions = {k: output_to_xr(v, y.coords) for k, v in predictions.items()}
preds_xr = dict_to_xr(predictions, dim_name="model")

Let's plot the predicted value of $Q1_c$ compared to the measured value

In [ ]:
%%opts Image[width=800, height=160, colorbar=True](cmap='inferno')

hv.Dataset(preds_xr.isel(x=0, y=8).Q1c).to.image(["time", "z"])\
.layout().cols(1)\
.redim.range(z=(0,20000))

Now, let's look at some scatter plots of the predicted vs the actual Q1c for different heights. The x and y axes are scaled so that 0 (1) corresponds to the minumum (maximum) actual Q1c for that height.

In [ ]:
(scatter_plot_z(preds_xr.Q1c, "slp", "true", "model").relabel("RNet")
+scatter_plot_z(preds_xr.Q1c, "rnet", "true", "model").relabel("RNet")
+ scatter_plot_z(preds_xr.Q1c, "linear", "true", "model").relabel("Linear Model")).cols(2)

The RNet and the modified single layer perceptron (SLP) both perform similarly well on this training dataset, but the linear model is much worse. In fact, the neural network fitted models, almost seem like a denoised version of the estimated heat source. Basically, it seems the Krasnopolsky result is the best option as far as deterministic parametrization is concerned. The main questions I have now are:


- Will these schemes perform well in a prognostic setting?
- Can we train these networks on the full NGAqua dataset including the tropics and sub-tropics? If not, what sort of network architectures will work well for that dataset?
- Are the answers very different when we train on a simulation with a different climate?